In [1]:
!pip install sumy
!pip install psycopg2-binary
!pip install pyspark
!pip install python-dotenv

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 9.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 15.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.2/775.2 kB 17.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 30.4 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=2

In [3]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, trim, regexp_extract, col, regexp_replace, split, lit, row_number, rand
from pyspark.sql.window import Window
from dotenv import load_dotenv

In [6]:
# Spark 세션 초기화
spark = SparkSession.builder \
    .appName("JSON to DataFrame") \
    .getOrCreate()

# JSON 파일 읽기
df = spark.read.option("multiline", "true").json("240524_codeit_final.json")

# CATEGORY 테이블에 삽입할 정보를 담은 df
df_category = df.select(col("big_categ").alias("category_name")) \
                .withColumn("platform_id", lit(3)) \
                .dropna()

# SUBCATEGORY 테이블에 삽입할 정보를 담은 df
df_subcategory = df.withColumn("sub_categs", explode(col("sub_categs"))) \
                .select(
                    col("big_categ").alias("category_name"),
                    col("sub_categs.sub_categ").alias("subcategory_name")
                ) \
                .dropna()

# COURSE 테이블에 삽입할 정보를 담은 df
df_course = df.withColumn("sub_categs", explode(col("sub_categs"))) \
                .withColumn("lectures", explode(col("sub_categs.lectures"))) \
                .select(
                    col("sub_categs.sub_categ").alias("subcategory_name"),
                    col("lectures.title").alias("course_title"),
                    col("lectures.summary").alias("summary"),
                    col("lectures.lecture_num").alias("num_of_lecture"),
                    col("lectures.link").alias("url")
                ) \
                .dropna()

# num_of_lecture에서 숫자 정보만 추출
df_course = df_course.withColumn("num_of_lecture", regexp_extract(col("num_of_lecture"), "\\d+", 0).cast("int")) 
df_course = df_course.dropDuplicates(["url"])

# 최종 데이터 확인
df_category.show()
df_subcategory.show()
df_course.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/24 14:00:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---------------+-----------+
|  category_name|platform_id|
+---------------+-----------+
|        웹 개발|          3|
|데이터 사이언스|          3|
|컴퓨터 사이언스|          3|
|프로그래밍 언어|          3|
|           기타|          3|
+---------------+-----------+

+---------------+--------------------+
|  category_name|    subcategory_name|
+---------------+--------------------+
|        웹 개발|          프론트엔드|
|        웹 개발|              백엔드|
|        웹 개발|              풀스택|
|데이터 사이언스|         데이터 분석|
|데이터 사이언스|           인공 지능|
|컴퓨터 사이언스|     프로그래밍 기초|
|컴퓨터 사이언스|   알고리즘·자료구조|
|컴퓨터 사이언스|객체 지향 프로그래밍|
|프로그래밍 언어|              Python|
|프로그래밍 언어|          JavaScript|
|           기타|        데이터베이스|
|           기타|         업무 자동화|
|           기타|           개발 도구|
|           기타|             IT 교양|
|           기타|              디자인|
+---------------+--------------------+

+-----------------+-----------------------------+-----------------------------------+--------------+--------------------+
| subcategory_name|    

In [7]:
# .env 파일 로드
load_dotenv()

db_url = os.getenv("DB_URL")
db_properties = {
    "driver": os.getenv("DB_DRIVER"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD")
}

In [8]:
# 데이터베이스에서 CATEGORY 정보 읽어와서 저장
df_category = spark.read.jdbc(url=db_url, table="CATEGORY", properties=db_properties)

# SUBCATEGORY 데이터프레임에 category_id 정보 매핑
df_subcategory = df_subcategory.join(df_category, df_subcategory.category_name == df_category.category_name, "left_outer") \
    .select(df_subcategory["*"], df_category.category_id.alias("category_id"))

df_subcategory = df_subcategory.select("category_id", "subcategory_name")

df_subcategory.show()

+-----------+--------------------+
|category_id|    subcategory_name|
+-----------+--------------------+
|          4|              Python|
|          4|          JavaScript|
|          5|        데이터베이스|
|          5|         업무 자동화|
|          5|           개발 도구|
|          5|             IT 교양|
|          5|              디자인|
|          1|          프론트엔드|
|          1|              백엔드|
|          1|              풀스택|
|          2|         데이터 분석|
|          2|           인공 지능|
|          3|     프로그래밍 기초|
|          3|   알고리즘·자료구조|
|          3|객체 지향 프로그래밍|
+-----------+--------------------+



In [9]:
# # 데이터베이스에 CATEGORY 정보 삽입
# df_category.write.jdbc(url=db_url, table="CATEGORY", mode="ignore", properties=db_properties)

spark.read.jdbc(url=db_url, table="CATEGORY", properties=db_properties).show()

# # 데이터베이스에 SUBCATEGORY 정보 삽입
# df_subcategory.write.jdbc(url=db_url, table="SUBCATEGORY", mode="ignore", properties=db_properties)

spark.read.jdbc(url=db_url, table="SUBCATEGORY", properties=db_properties).show()

+-----------+---------------+-----------+
|category_id|  category_name|platform_id|
+-----------+---------------+-----------+
|          1|        웹 개발|          3|
|          2|데이터 사이언스|          3|
|          3|컴퓨터 사이언스|          3|
|          4|프로그래밍 언어|          3|
|          5|           기타|          3|
|         -1|           None|          2|
+-----------+---------------+-----------+

+--------------+--------------------+-----------+
|subcategory_id|    subcategory_name|category_id|
+--------------+--------------------+-----------+
|             1|              Python|          4|
|             2|          JavaScript|          4|
|             3|        데이터베이스|          5|
|             4|         업무 자동화|          5|
|             5|           개발 도구|          5|
|             6|             IT 교양|          5|
|             7|              디자인|          5|
|             8|          프론트엔드|          1|
|             9|              백엔드|          1|
|            10|              풀스택

In [10]:
df_subcategory = spark.read.jdbc(url=db_url, table="SUBCATEGORY", properties=db_properties)
df_subcategory.show()

df_course = df_course.join(df_subcategory, df_course.subcategory_name == df_subcategory.subcategory_name, "left_outer") \
    .select(df_course["*"], df_subcategory.subcategory_id.alias("subcategory_id"))

df_course.show()

+--------------+--------------------+-----------+
|subcategory_id|    subcategory_name|category_id|
+--------------+--------------------+-----------+
|             1|              Python|          4|
|             2|          JavaScript|          4|
|             3|        데이터베이스|          5|
|             4|         업무 자동화|          5|
|             5|           개발 도구|          5|
|             6|             IT 교양|          5|
|             7|              디자인|          5|
|             8|          프론트엔드|          1|
|             9|              백엔드|          1|
|            10|              풀스택|          1|
|            11|         데이터 분석|          2|
|            12|           인공 지능|          2|
|            13|     프로그래밍 기초|          3|
|            14|   알고리즘·자료구조|          3|
|            15|객체 지향 프로그래밍|          3|
|            -1|                None|         -1|
+--------------+--------------------+-----------+

+-----------------+-----------------------------+--------------

In [11]:
# 모든 컬럼 이름 가져오기
all_columns = df_course.columns

# "subcategory_name"을 제외한 컬럼 이름 리스트 만들기
columns_to_select = [col for col in all_columns if col != "subcategory_name"]

# 해당 컬럼들을 선택하여 새로운 데이터프레임 생성
df_course = df_course.select(*columns_to_select)

# 데이터베이스에 COURSE 정보 삽입
df_course.write.jdbc(url=db_url, table="COURSE", mode="ignore", properties=db_properties)

In [12]:
# section, review에 course_id 매핑
df_course_DB = spark.read.jdbc(url=db_url, table="COURSE", properties=db_properties).select("course_id", "url")


df_sectionInfo = df.withColumn("sub_categs", explode(col("sub_categs"))) \
                .withColumn("lectures", explode(col("sub_categs.lectures"))) \
                .select(
                    col("lectures.link").alias("url"),
                    col("lectures.curriculum").alias("curriculum")
                ) \
                .dropna()

df_reviewInfo = df.withColumn("sub_categs", explode(col("sub_categs"))) \
                .withColumn("lectures", explode(col("sub_categs.lectures"))) \
                .select(
                    col("lectures.link").alias("url"),
                    col("lectures.review").alias("review_info")
                ) \
                .dropna()

df_sectionInfo_withID = df_sectionInfo.join(df_course_DB, on="url", how="inner").drop("url").distinct()
df_reviewInfo_withID = df_reviewInfo.join(df_course_DB, on="url", how="inner").drop("url").distinct()

df_sectionInfo_withID.show(truncate=True)
df_reviewInfo_withID.show()

+--------------------------------+---------+
|                      curriculum|course_id|
+--------------------------------+---------+
|      1\n토픽 시작하기\nTypeS...|      126|
|            1\nStyled Compone...|      117|
|  1\n데이터 모델링이란?\n데이...|      120|
|        1\n선형 회귀 (Linear ...|      151|
|    1\n모듈\n모듈이란?\n모듈 ...|      171|
| 1\n시각화와 그래프\n시각화의...|      142|
|       1\n유저 기능과 django-...|      131|
|        1\n결정 트리 (Decisio...|      150|
|  1\n인공 신경망이란\n인공 신...|      149|
|  1\n클라우드 컴퓨팅이란?\n클...|      157|
|     1\nCSS 레이아웃\n토픽 소...|      103|
|       1\n수강 가이드\nNext.j...|      101|
|  1\n프로그래밍 혁명\n코딩이 ...|      161|
| 1\n컴파일러와 인터프리터\n다...|      159|
|  1\n수업 소개\n웹사이트는 어...|      108|
|  1\n자바스크립트 첫 걸음\n자...|      122|
|1\n인터랙티브 자바스크립트 시...|      107|
| 1\n알고리즘이란?\n알고리즘이...|      179|
|  1\n유닉스 커맨드 시작하기\n...|      128|
|  1\n파이썬 환경 기본\n환경이...|      170|
+--------------------------------+---------+
only showing top 20 rows

+----------------------------------+---------+
|          

In [13]:
# df
df_sectionInfo_withID = df_sectionInfo_withID.withColumn(
    "curriculum",
    regexp_replace(col("curriculum"), r'^\n+', '')
)

df_parsed = df_sectionInfo_withID.withColumn(
    "section_split",
    split(col("curriculum"), r'\n(?=\d+\n)')
)

df_exploded = df_parsed.withColumn(
    "section",
    explode(col("section_split"))
).filter(trim(col("section")) != "")

df_exploded = df_exploded.withColumn(
    "section_num",
    regexp_extract(col("section"), r'^(\d+)', 1)
).withColumn(
    "section_title",
    regexp_extract(col("section"), r'^\d+\n([^\n]+)', 1)
)

df_exploded = df_exploded.withColumn(
    "subsection",
    split(
        regexp_replace(col("section"), r'^\d+\n[^\n]+\n', ''),
        '\n'
    )
).withColumn(
    "subsection",
    explode(col("subsection"))
).filter(trim(col("subsection")) != "")

df_result = df_exploded.select(
    col("course_id"),
    col("section_num").cast("int"),
    col("section_title").alias("section_name"),
    col("subsection").alias("subsection_name")
).orderBy("course_id", "section_num")

windowSpec = Window.partitionBy("course_id", "section_num").orderBy("subsection_name")

df_result_with_subsection_num = df_result.withColumn(
    "subsection_num",
    row_number().over(windowSpec)
)

df_section = df_result_with_subsection_num.select("course_id", "section_num", "section_name").distinct()
df_subsection = df_result_with_subsection_num.select("course_id", "section_num", "subsection_num", "subsection_name")

df_section.show(truncate=True)
df_subsection.show(truncate=True)

+---------+-----------+------------------------+
|course_id|section_num|            section_name|
+---------+-----------+------------------------+
|      167|          1|       자바스크립트 기초|
|      148|          1|          웹의 기본 요소|
|      163|          4|회로 설계(Circuit des...|
|      124|          1|                    객체|
|      133|          3|             경로 다루기|
|      176|          1|   알고리즘 연습 Level 1|
|      147|          3|             프로젝트 II|
|      154|          1|        추천 시스템이란?|
|      122|          2|    프로그래밍 기본 개념|
|      128|          2|  디렉토리와 파일 다루기|
|      152|          1|           데이터 전처리|
|      157|          1|    클라우드 컴퓨팅이란?|
|      163|          2|   컴퓨터의 언어, 이진법|
|      137|          1|           객체와 클래스|
|      108|          4|                완성하기|
|      141|          3|          좋은 UI 디자인|
|      175|          2|             그래프 탐색|
|      139|          1|           토픽 시작하기|
|      166|          2|    프로그래밍 기본 개념|
|      133|          4|             폴더 다루기

In [14]:
df_section.write.jdbc(url=db_url, table="SECTION", mode="ignore", properties=db_properties)
df_subsection.write.jdbc(url=db_url, table="SUBSECTION", mode="ignore", properties=db_properties)

spark.read.jdbc(url=db_url, table="SECTION", properties=db_properties).show()
spark.read.jdbc(url=db_url, table="SUBSECTION", properties=db_properties).show()

+---------+-----------+----------------------------------+--------------+------------+----------+----------+
|course_id|section_num|                      section_name|num_of_lecture|running_time|updated_at|created_at|
+---------+-----------+----------------------------------+--------------+------------+----------+----------+
|      202|          1|                      오리엔테이션|          null|        null|2024-05-24|2024-05-23|
|      202|          2|              나만의 나침반 만들기|          null|        null|2024-05-24|2024-05-23|
|      202|          3|  작은 브랜드를 위한 비주얼 가이드|          null|        null|2024-05-24|2024-05-23|
|      202|          4|복잡한 디자인을 깔끔하게 바꿔주...|          null|        null|2024-05-24|2024-05-23|
|      202|          5|     에이핫이 인생을 디자인한 방법|          null|        null|2024-05-24|2024-05-23|
|      190|          1|            HTML/CSS/JS 기초 : ...|          null|        null|2024-05-24|2024-05-23|
|      190|          2|             Git을 활용한 버전관리|          null|        null

In [21]:
# 리뷰 분리
df_reviews_split = df_reviewInfo_withID.withColumn(
    "review",
    explode(split(col("review_info"), r'\n(?=\w+\n)'))
).filter(trim(col("review")) != "")

# 유저 이름과 리뷰 분리, 리뷰만 선택
df_reviews_cleaned = df_reviews_split.withColumn(
    "review_text",
    regexp_extract(col("review"), r'^\w+\n(.*)', 1)
).filter(trim(col("review_text")) != "")

# 필요한 컬럼 선택 및 정렬
df_reviews_result = df_reviews_cleaned.select(
    col("course_id"),
    col("review_text").alias("summary")
).orderBy("course_id")

# 윈도우 함수 정의
window_spec = Window.partitionBy("course_id").orderBy(rand())

# 각 course_id 당 3개의 랜덤한 리뷰 선택
df_review = df_reviews_result.withColumn(
    "rank",
    row_number().over(window_spec)
).filter(col("rank") <= 3).drop("rank").distinct()


# 결과 출력
df_review.show(truncate=False)

+---------+---------------------------------------------------------------------------------------------------------------+
|course_id|summary                                                                                                        |
+---------+---------------------------------------------------------------------------------------------------------------+
|100      |초보자인 저에게 어려울 것 같았지만  전반적인 내용을 알기 편하게 설명해주셔서 좋았습니다.                       |
|100      |간단하게 react 관련 사이트 제작을 맛볼수 있는 소중한 경험이였습니다.                                           |
|100      |실제 서비스 만드는 과정에 큰 도움이 되는 강의 입니다                                                           |
|104      |wow 재밌어요 리액트 너무 쉽네요                                                                                |
|104      |일반 자바스크립트랑 좀 접근 개념이 달라 힘들었지만! 너무 교육이 잘장리되있어서 다 습득이 됬습니다!             |
|104      |리액트의 장점이 무엇인지 리액트를 어떻게 제대로 사용하는지 배울수있었습니다                                    |
|105      |시간날 떄 다시 들어보겠습니다~ react강의 준비 하시느라 고생하셨네요~       

In [23]:
df_review.write.jdbc(url=db_url, table="REVIEW_SUMMARY", mode="ignore", properties=db_properties)

spark.read.jdbc(url=db_url, table="REVIEW_SUMMARY", properties=db_properties).show()

+---------+-----------------------------------+----------+----------+
|course_id|                            summary|updated_at|created_at|
+---------+-----------------------------------+----------+----------+
|      100|  초보자인 저에게 어려울 것 같았...|2024-05-24|2024-05-24|
|      100|      간단하게 react 관련 사이트...|2024-05-24|2024-05-24|
|      100|   실제 서비스 만드는 과정에 큰 ...|2024-05-24|2024-05-24|
|      104|    wow 재밌어요 리액트 너무 쉽네요|2024-05-24|2024-05-24|
|      104|  일반 자바스크립트랑 좀 접근 개...|2024-05-24|2024-05-24|
|      104| 리액트의 장점이 무엇인지 리액트...|2024-05-24|2024-05-24|
|      105|  시간날 떄 다시 들어보겠습니다~...|2024-05-24|2024-05-24|
|      105|  짤막하게 유용한 내용만 쏙쏙 학...|2024-05-24|2024-05-24|
|      107|  잘 만들어주신 실습 문제들 덕에...|2024-05-24|2024-05-24|
|      107| 자바스크립트를 매우 세밀하게 배...|2024-05-24|2024-05-24|
|      107| 지식이 쑥쑥 올라가는속도가 들립...|2024-05-24|2024-05-24|
|      115| 반응형 웹이 어떻게 만들어지는지...|2024-05-24|2024-05-24|
|      115|   반응형에 대한 이해 자체는 쉽지만|2024-05-24|2024-05-24|
|      115|  기본 개념을 이해하는데 좋아요....|2024-05-24|202

In [ ]:
# Spark 세션 종료
spark.stop()